<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-dependencies" data-toc-modified-id="Load-dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load dependencies</a></span></li><li><span><a href="#Read-csv-files-to-DFs" data-toc-modified-id="Read-csv-files-to-DFs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read csv files to DFs</a></span></li><li><span><a href="#Fill-missing-values" data-toc-modified-id="Fill-missing-values-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fill missing values</a></span></li><li><span><a href="#Add-date-part" data-toc-modified-id="Add-date-part-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Add date part</a></span></li><li><span><a href="#Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc." data-toc-modified-id="Add-extra-columns-(purchased-on-weekend,-monthend,-month_diff-etc.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Add extra columns (purchased on weekend, monthend, month_diff etc.</a></span></li><li><span><a href="#Drop-unnecessary-date-cols" data-toc-modified-id="Drop-unnecessary-date-cols-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Drop unnecessary date cols</a></span></li><li><span><a href="#One-hot-encoding-category_1-&amp;-authorized-(not-needed)" data-toc-modified-id="One-hot-encoding-category_1-&amp;-authorized-(not-needed)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>One hot encoding category_1 &amp; authorized (not needed)</a></span></li><li><span><a href="#Aggregate-by-card_id" data-toc-modified-id="Aggregate-by-card_id-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Aggregate by card_id</a></span></li><li><span><a href="#Aggregate-by-month-on-card_id-on-purchase_amount-(not-working)" data-toc-modified-id="Aggregate-by-month-on-card_id-on-purchase_amount-(not-working)-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Aggregate by month on card_id on purchase_amount (not working)</a></span></li><li><span><a href="#Aggregates-on-categories" data-toc-modified-id="Aggregates-on-categories-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Aggregates on categories</a></span></li><li><span><a href="#Add-extra-cols-on-agg" data-toc-modified-id="Add-extra-cols-on-agg-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Add extra cols on agg</a></span></li><li><span><a href="#Load-test-&amp;-train-DFs" data-toc-modified-id="Load-test-&amp;-train-DFs-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Load test &amp; train DFs</a></span></li><li><span><a href="#Add-date-part-to-test-&amp;-train-dfs" data-toc-modified-id="Add-date-part-to-test-&amp;-train-dfs-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>Add date part to test &amp; train dfs</a></span></li><li><span><a href="#Remove-extra-date-cols" data-toc-modified-id="Remove-extra-date-cols-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>Remove extra date cols</a></span></li><li><span><a href="#Merge-train-&amp;-test-with-new-&amp;-old-transactions-history" data-toc-modified-id="Merge-train-&amp;-test-with-new-&amp;-old-transactions-history-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>Merge train &amp; test with new &amp; old transactions history</a></span></li><li><span><a href="#Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc" data-toc-modified-id="Add-extra-columns-like-age,-total-transactions,-installments,-purchase-amount,-first-buy-etc-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>Add extra columns like age, total transactions, installments, purchase amount, first buy etc</a></span></li><li><span><a href="#Mark-the-outliers" data-toc-modified-id="Mark-the-outliers-17"><span class="toc-item-num">17&nbsp;&nbsp;</span>Mark the outliers</a></span></li><li><span><a href="#Target-encode-the-outliers" data-toc-modified-id="Target-encode-the-outliers-18"><span class="toc-item-num">18&nbsp;&nbsp;</span>Target encode the outliers</a></span></li></ul></div>

1. Load dependencies
2. Read csv files to dataframes
3. Fill missing values
4. Add date part 
5. Add extra columns (purchased on weekend, monthend, month_diff etc.
6. Aggregate by card_id
7. Aggregate by categories
8. Mark categorical columns
9. Add exta interpreted columns on aggregates
10. Load test & train csvs to dfs
11. Add date part to test & train dfs
12. Merge train & test with new & old transactions history
13. Add extra columns like age, total transactions, installments, purchase amount, first buy etc
14. Mark the outliers
15. Target encode the outliers 
16. Save to feather

### Load dependencies

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 4
%autosave 120

Autosaving every 120 seconds


In [2]:
from fastai.io import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from pandas_summary import DataFrameSummary
from IPython.display import display
from sklearn import metrics
import feather

### Read csv files to DFs

In [3]:
PATH = '../data/elo/'

In [4]:
files = ['historical_transactions', 'new_merchant_transactions']

In [5]:
hist_trans, new_hist_trans = [pd.read_csv(f'{PATH}{c}.csv') for c in files]

### Fill missing values

In [6]:
def fill_nas_for_transactions_df(df):
    # Fill nas for category_3 with mode
    df['category_2'].fillna(1.0,inplace=True)
    df['category_3'].fillna('A',inplace=True)
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    return df

In [7]:
dfs = [hist_trans, new_hist_trans]

In [8]:
hist_trans, new_hist_trans = [fill_nas_for_transactions_df(df) for df in dfs]

### Add date part

In [9]:
add_datepart(hist_trans, 'purchase_date', drop=False)
add_datepart(new_hist_trans, 'purchase_date', drop=False)

### Add extra columns (purchased on weekend, monthend, month_diff etc.

In [10]:
def add_extra_cols(df):
    df['purchased_on_weekend'] = (df.purchase_Dayofweek >=5).astype(int)
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    return df

In [11]:
hist_trans, new_hist_trans = [add_extra_cols(df) for df in dfs]

In [12]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

### Drop unnecessary date cols

In [13]:
new_hist_trans.columns.values

array(['authorized_flag', 'card_id', 'city_id', 'category_1', 'installments', 'category_3',
       'merchant_category_id', 'merchant_id', 'month_lag', 'purchase_amount', 'purchase_date', 'category_2',
       'state_id', 'subsector_id', 'purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day',
       'purchase_Dayofweek', 'purchase_Dayofyear', 'purchase_Is_month_end', 'purchase_Is_month_start',
       'purchase_Is_quarter_end', 'purchase_Is_quarter_start', 'purchase_Is_year_end',
       'purchase_Is_year_start', 'purchase_Elapsed', 'purchased_on_weekend', 'month_diff'], dtype=object)

In [14]:
def drop_extra_date_cols(df):
    cols_to_drop = ['purchase_Year', 'purchase_Month', 'purchase_Week', 'purchase_Day',
       'purchase_Dayofweek', 'purchase_Dayofyear', 'purchase_Is_month_end', 'purchase_Is_month_start',
       'purchase_Is_quarter_end', 'purchase_Is_quarter_start', 'purchase_Is_year_end',
       'purchase_Is_year_start']
    return df.drop(cols_to_drop, axis=1)

In [15]:
# hist_trans, new_hist_trans = [drop_extra_date_cols(df) for df in dfs]

In [16]:
hist_trans.shape, new_hist_trans.shape

((29112361, 29), (1963031, 29))

### One hot encoding category_1 & authorized (not needed)

In [17]:
def one_hot_encode(df):
    return pd.get_dummies(df, columns=['authorized_flag'])

In [18]:
# hist_trans, new_hist_trans = [one_hot_encode(df) for df in dfs]

In [19]:
hist_trans.to_feather('hist_trans')
new_hist_trans.to_feather('new_hist_trans')
hist_trans = feather.read_dataframe('hist_trans')
new_hist_trans = feather.read_dataframe('new_hist_trans')

### Aggregate by card_id

In [29]:
def aggregate_by_card_id(df):
    unique_cols = ['city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'subsector_id']
    aggs = {}
    for c in unique_cols:
        aggs[c] = ['nunique'] 
    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min', np.ptp]
    aggs['month_lag'] = ['max','min','mean','var']
    aggs['month_diff'] = ['mean', 'std', 'var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['purchased_on_weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    
    for col in ['category_2','category_3']:
        df[col+'_amount_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        aggs[col+'_amount_mean'] = ['mean']
            
    new_df = df.groupby(['card_id']).agg(aggs)
    new_df.columns = ['_'.join(col).strip() for col in new_df.columns.values]
    new_df.reset_index(inplace=True)
    other_df = (df.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    new_df = pd.merge(other_df, new_df, on='card_id', how='left')
    new_df["purchase_date_ptp"] = new_df["purchase_date_ptp"].dt.days
    new_df['purchase_date_diff'] = (new_df['purchase_date_max'] - new_df['purchase_date_min']).dt.days
    new_df['purchase_date_average'] = new_df['purchase_date_diff']/new_df['card_id_size']
    new_df['purchase_date_uptonow'] = (datetime.datetime.today() - new_df['purchase_date_max']).dt.days
    return new_df

In [30]:
import gc
gc.collect()

28

In [31]:
%time hist_trans_agg, new_hist_trans_agg = [aggregate_by_card_id(df) for df in [hist_trans, new_hist_trans]]

CPU times: user 2min 44s, sys: 12.3 s, total: 2min 57s
Wall time: 2min 15s


### Aggregate by month on card_id on purchase_amount (not working)

In [32]:
def aggregate_per_month(history):
    grouped = history.groupby(['card_id', 'month_diff'])

    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'var'],
            'installments': ['count', 'sum', 'mean', 'max'],
            }

    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean'])
    final_group.columns = ['_s'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group

In [33]:
# [hist_trans_agg_s, new_hist_trans_agg_s] = [aggregate_per_month(df) for df in [hist_trans, 
#                                                                              new_hist_trans]]

In [34]:
# for df in [hist_trans_agg_s, new_hist_trans_agg_s]:
#     df.drop('month_diff_smean', inplace=True, axis=1)

In [35]:
# hist_trans_agg_s.head().T

In [36]:
hist_trans_agg.shape, new_hist_trans_agg.shape

((325540, 39), (290001, 39))

### Aggregates on categories

In [37]:
def agg_on_cat(df, category, feature):
    temp_df = df.pivot_table(index='card_id', columns=category, aggfunc={feature: ['sum', 'mean']})
    cols = [category + '_{0[2]}_{0[0]}_{0[1]}'.format(col) for col in temp_df.columns.tolist()]
    temp_df.columns = cols
    return temp_df

In [38]:
def get_cat_agg(df):
    agg_df = agg_on_cat(df, 'category_1', 'purchase_amount')
#     agg_df = pd.merge(agg_df, agg_on_cat(df, 'category_2', 'purchase_amount'), on='card_id', how='left')
#     agg_df = pd.merge(agg_df, agg_on_cat(df, 'category_3', 'purchase_amount'), on='card_id', how='left')
    agg_df = pd.merge(agg_df, agg_on_cat(df, 'authorized_flag', 'purchase_amount'), on='card_id', how='left')
#     agg_df.fillna(0, inplace=True)
    return agg_df

In [39]:
%time hist_trans_agg_cat, new_hist_trans_agg_cat = [get_cat_agg(df) for df in [hist_trans, new_hist_trans]]

CPU times: user 8.54 s, sys: 4.8 s, total: 13.3 s
Wall time: 6.84 s


In [40]:
hist_trans_agg_cat.shape, new_hist_trans_agg_cat.shape

((325540, 8), (290001, 6))

In [41]:
new_hist_trans_agg_cat.head().T

card_id,C_ID_00007093c1,C_ID_0001238066,C_ID_0001506ef0,C_ID_0001793786,C_ID_000183fdda
category_1_0_purchase_amount_mean,-0.664262,-0.564558,-0.723677,-0.007407,-0.599162
category_1_1_purchase_amount_mean,NaN,-0.650332,NaN,NaN,NaN
category_1_0_purchase_amount_sum,-1.328524,-13.549391,-1.447354,-0.229620,-6.590778
category_1_1_purchase_amount_sum,NaN,-1.300665,NaN,NaN,NaN
authorized_flag_1_purchase_amount_mean,-0.664262,-0.571156,-0.723677,-0.007407,-0.599162
authorized_flag_1_purchase_amount_sum,-1.328524,-14.850055,-1.447354,-0.229620,-6.590778


In [42]:
hist_trans_agg.shape, new_hist_trans_agg.shape

((325540, 39), (290001, 39))

### Add extra cols on agg

In [43]:
def add_extra_cols_on_agg(df):
    df['inverse_avg_transactions_per_day'] = df['purchase_date_diff']/df['card_id_size']
    df['days_since_last_transaction'] = (datetime.datetime.today() - df['purchase_date_max']).dt.days
    df['repurchase_merchant_rate'] = df['transactions_count']/df['merchant_id_nunique']
    df['merchant_category_repurchase'] = df['merchant_category_id_nunique']/df['merchant_id_nunique']
    df['avg_spend_per_merchant'] = df['purchase_amount_sum']/df['merchant_id_nunique']
    df['avg_trans_per_merchant'] = df['transactions_count']/df['merchant_id_nunique']
    df['avg_spend_per_transaction'] = df['purchase_amount_sum']/df['transactions_count']
    return df

In [44]:
[hist_trans_agg, new_hist_trans_agg] = [add_extra_cols_on_agg(df) for df in [hist_trans_agg, 
                                                                             new_hist_trans_agg]]

In [45]:
hist_trans_agg.to_feather('hist_trans_agg_n')
new_hist_trans_agg.to_feather('new_hist_trans_agg_n')

In [46]:
hist_trans_agg = feather.read_dataframe('hist_trans_agg_n')
new_hist_trans_agg = feather.read_dataframe('new_hist_trans_agg_n')

### Load test & train DFs

In [47]:
PATH = 'data/elo/'

In [48]:
train, test = [pd.read_csv(f'{PATH}{c}') for c in ['train.csv', 'test.csv']]

### Add date part to test & train dfs

In [49]:
add_datepart(train, 'first_active_month', drop=False)
add_datepart(test, 'first_active_month', drop=False)

### Remove extra date cols

In [50]:
train.columns.values

array(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target',
       'first_active_monthYear', 'first_active_monthMonth', 'first_active_monthWeek',
       'first_active_monthDay', 'first_active_monthDayofweek', 'first_active_monthDayofyear',
       'first_active_monthIs_month_end', 'first_active_monthIs_month_start',
       'first_active_monthIs_quarter_end', 'first_active_monthIs_quarter_start',
       'first_active_monthIs_year_end', 'first_active_monthIs_year_start', 'first_active_monthElapsed'],
      dtype=object)

In [51]:
def drop_extra_date_cols_2(df):
    cols_to_drop = ['first_active_monthYear', 'first_active_monthMonth', 'first_active_monthWeek',
       'first_active_monthDay', 'first_active_monthDayofweek', 'first_active_monthDayofyear',
       'first_active_monthIs_month_end', 'first_active_monthIs_month_start',
       'first_active_monthIs_quarter_end', 'first_active_monthIs_quarter_start',
       'first_active_monthIs_year_end', 'first_active_monthIs_year_start']
    return df.drop(cols_to_drop, axis=1)

In [52]:
# [train, test] = [drop_extra_date_cols_2(df) for df in [train, test]]

### Merge train & test with new & old transactions history

In [53]:
def join_dfs(left, right, left_on, right_on=None, suffix='_old'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, suffixes=("", suffix))

In [54]:
train_df = join_dfs(train, new_hist_trans_agg, left_on='card_id')

In [55]:
train_df = join_dfs(train_df, hist_trans_agg, left_on='card_id', suffix='_old')

In [56]:
train_df.shape

(201917, 109)

In [57]:
# train_df = join_dfs(train_df, hist_trans_agg_cat, left_on='card_id', suffix='_old')

In [58]:
test_df = join_dfs(test, new_hist_trans_agg, left_on='card_id')
test_df = join_dfs(test_df, hist_trans_agg, left_on='card_id', suffix='_old')
# test_df = join_dfs(test_df, new_hist_trans_agg_cat, left_on='card_id')

In [59]:
test_df.shape, train_df.shape

((123623, 108), (201917, 109))

### Add extra columns like age, total transactions, installments, purchase amount, first buy etc

In [60]:
for df in [train_df, test_df]:
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['card_id_total'] = df['card_id_size']+df['card_id_size_old']
    df['purchase_amount_total'] = df['purchase_amount_sum']+df['purchase_amount_sum_old']
    df['installments_total'] = df['installments_sum'] + df['installments_sum_old']
    df['hist_first_buy'] = (df['purchase_date_min_old'] - df['first_active_month']).dt.days
    df['new_first_buy'] = (df['purchase_date_min'] - df['first_active_month']).dt.days
    df['avg_spend_per_transaction'] = df['purchase_amount_total']/df['card_id_total']

### Mark the outliers

In [61]:
train_df['outliers'] = 0
train_df.loc[train_df['target'] < -31, 'outliers'] = 1
train_df['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

### Target encode the outliers 

In [62]:
train_df.shape, test_df.shape

((201917, 116), (123623, 114))

In [63]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train_df.groupby([f])['outliers'].mean()
    train_df[f] = train_df[f].map(order_label)
    test_df[f] = test_df[f].map(order_label)

In [64]:
train_df.to_feather('train_df')
test_df.to_feather('test_df')

In [ ]:
# len(np.unique(train_df.columns.values))

In [ ]:
# u, c = np.unique(train_df.columns.values, return_counts=True)

In [ ]:
# u[c > 1]

In [65]:
train_df.columns.values

array(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3', 'target',
       'first_active_monthYear', 'first_active_monthMonth', 'first_active_monthWeek',
       'first_active_monthDay', 'first_active_monthDayofweek', 'first_active_monthDayofyear',
       'first_active_monthIs_month_end', 'first_active_monthIs_month_start',
       'first_active_monthIs_quarter_end', 'first_active_monthIs_quarter_start',
       'first_active_monthIs_year_end', 'first_active_monthIs_year_start', 'first_active_monthElapsed',
       'transactions_count', 'city_id_nunique', 'merchant_category_id_nunique', 'merchant_id_nunique',
       'state_id_nunique', 'subsector_id_nunique', 'purchase_amount_sum', 'purchase_amount_max',
       'purchase_amount_min', 'purchase_amount_mean', 'purchase_amount_var', 'installments_sum',
       'installments_max', 'installments_min', 'installments_mean', 'installments_var', 'purchase_date_max',
       'purchase_date_min', 'purchase_date_ptp', 'month_lag_max

In [66]:
train_df[['elapsed_time']]

,elapsed_time
0,598
1,749
2,902
3,506
4,445
5,871
6,780
7,506
8,537
9,902


In [ ]:
# test_df.columns.values

In [ ]:
train_df.drop(['authorized_flag_0_purchase_amount_mean', 'authorized_flag_0_purchase_amount_sum'], axis=1, inplace=True)

In [ ]:
train_df.to_feather('train_df')
test_df.to_feather('test_df')